# API. Estudio de las variables 🪢

En este jupyter notebook se encuentra todo el proceso de exportación de datos para el estudio de las variables requeridas, utilizando la API Foursquare. **Para la empresa LiveStream**.

*Consideraciones*
- Se ha establecido un "limit" de 10 elementos al realizar las llamadas a la API, para facilitar la selección, además de no superar el límite de llamadas a la API, permitida en la versión gratuita.

## Índice 📎

1. Importación de librerías y funciones
2. Importación de los datos
3. Conexión con la API
4. Estudio de las variables
>- Aeropuertos
>- Discotecas
>- Starbucks
>- Restaurantes veganos
>- Estación de autobuses
>- Colegios
>- Veterinarios

## 1. Importación de librerías y funciones 📚

In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import geopandas as gpd
from pymongo import MongoClient,GEOSPHERE
from functools import reduce
import operator
from cartoframes.viz import Map, Layer, popup_element
from pymongo import MongoClient
import shapely.geometry
import math

Otras funciones:

In [2]:
#import Funciones Funciones as fu

## 2. Importación de los datos 📍

Para extraer las coordenadas de la oficina tenemos dos posibilidades, o bien mediante la información que nos proporciona la base de datos o bien a través de la API de geocode, introduciendo la dirección de la oficina selecionada.

In [3]:
#Importación del dataset con los datos de las oficinas
oficinas = pd.read_csv("./oficinas_info.csv")

In [4]:
#Indicamos la ciudad donde se sitúa la oficina
city = "New York"

In [5]:
data = requests.get(f"https://geocode.xyz/{city}?json=1").json()
data

{'standard': {'addresst': {},
  'city': 'New York',
  'prov': 'US',
  'countryname': 'United States of America',
  'postal': {},
  'confidence': '0.90'},
 'longt': '-73.95861',
 'alt': {'loc': {'longt': '-73.9536869565217',
   'prov': 'NY',
   'city': 'NEW YORK',
   'postal': '10075',
   'score': '25',
   'latt': '40.7724386956522'}},
 'elevation': {},
 'latt': '40.68908'}

In [6]:
#Creamos una función que saque las coordenadas de una dirección
def geocode(direccion): 
    """
    Saca las coordenadas de una dirección
    """
    data = requests.get(f"https://geocode.xyz/{direccion}?json=1").json()
    try:
        return {
            "type":"Point",
            "coordinates":  [float(data["latt"]), float(data["longt"])]
        }
    except:
        return data

In [7]:
#Creamos una variable que contenga la dirección de la oficina deseada
direccion = "636 Broadway, New York"

In [8]:
#Imprimimos las coordenadas en "type": "Point" para más tarde poder realizar la visualización y el tratamiento de datos.
coordenadas = geocode(direccion)
coordenadas

{'type': 'Point', 'coordinates': [40.72628, -73.99587]}

In [9]:
#Comprobamos que coinciden con los datos del DataFrame
oficinas

,Unnamed: 0,city,country_code,latitude,longitude,name,category_code,number_of_employees,founded_year
0,0,New York,USA,40.726155,-73.995625,Livestream,games_video,120,2007
1,1,New York,USA,40.717248,-74.002662,Boonty,games_video,150,2001


*Consideraciones: Se tomaran las coordenadas 40.726155, -73.995625 de la emprea LiveStream para el estudio de las varibles*

## 3. Conexión con la API. Foursquare 🔌

A continuación antes de realizar el estudio de las demás variables y requisitos que debe cumplir la nueva sede procederemos a establecer la conexión con la API de Foursquare, un servicio que proporciona la geolocalización de lugares específicos gracias a la recolección de datos por parte de los usuarios y consumidores de la propia app.

In [10]:
url = "https://api.foursquare.com/v2/venues/explore"

In [11]:
client_id = open("./client_id.txt").readlines()[0].rstrip()

In [12]:
client_secret = open("./client_secret.txt").readlines()[0].rstrip()

In [13]:
coordenadas = "40.726155, -73.995625"

## 4. Estudio de las variables ✳️

Una vez que tenemos la conexión con la API vamos a comprobar que la oficina se sitúa en una localización óptima en base a los requisitos establecidos por nuestros empleados:
- Aeropuertos y estaciones de autobuses para viajes a largas distancia y rutinarios
- En lo respectivo al ocio: discotecas, restaurantes veganos y Starbucks
- Colegios
- Veterinario

## 4.1 Aeropuerto ✈️

In [14]:
params1 = {"client_id" : client_id,
              "client_secret" : client_secret,
              "v": "20180323",
              "ll": coordenadas,
              "query": "airport",
              "limit": 10,
              "radius": 5000
             }
response1 = requests.get(url,params=params1).json()

Tras pasar las queries y los parámetros por la API, obtenemos un diccionario de con dos keys "meta" y "response", lod datos que nosostros necesitamos se encuentran en "response".

In [15]:
data1 = response1.get("response")

In [16]:
decode1 = data1.get("groups")[0] 

*Consideraciones: La razón por la que nos quedamos con el elemento 0 es porque los valores que quiero tomar se encuentran en una lista dentro del diccionarario (data1) anterior. Lo siguiente será extraer el nombre y las coordenadas, creando de nuevo una variable:*

In [17]:
aeropuerto = decode1.get("items")

In [18]:
mapa_nombre = ["venue", "name"]
mapa_latitud = ["venue","location","lat"]
mapa_longitud = ["venue","location","lng"]

In [19]:
#Fuentes
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [20]:
#Extración de los datos
aeropuertos = []
for dic in aeropuerto:
    lista = {}
    lista["nombre"] = getFromDict(dic, mapa_nombre)
    lista["latitud"] = getFromDict(dic, mapa_latitud)
    lista["longitud"] = getFromDict(dic, mapa_longitud)
    aeropuertos.append(lista)

In [21]:
#Creación del DataFrame
df_aeropuertos = pd.DataFrame(aeropuertos)
df_aeropuertos

,nombre,latitud,longitud
0,PH Helipad,40.732918,-73.995468
1,Corporate Express Travel,40.722429,-74.002404
2,Priority One Jets,40.709288,-74.006822
3,Atlantic Aviation,40.744048,-73.973925
4,Us Helicopter,40.702437,-74.013122
5,New York City Limo Service,40.745192,-73.983371
6,New Bell Car Service,40.693717,-73.960250
7,Times Square Limousine,40.752182,-74.003348
8,Charter Jet One,40.752684,-73.975902
9,Lower East Side Car Service,40.726151,-73.977334


In [22]:
#Lo transformamos en GeoDataFrame
gdf_aeropuertos = gpd.GeoDataFrame(df_aeropuertos, geometry = gpd.points_from_xy(df_aeropuertos.longitud, df_aeropuertos.latitud))
gdf_aeropuertos

,nombre,latitud,longitud,geometry
0,PH Helipad,40.732918,-73.995468,POINT (-73.99547 40.73292)
1,Corporate Express Travel,40.722429,-74.002404,POINT (-74.00240 40.72243)
2,Priority One Jets,40.709288,-74.006822,POINT (-74.00682 40.70929)
3,Atlantic Aviation,40.744048,-73.973925,POINT (-73.97393 40.74405)
4,Us Helicopter,40.702437,-74.013122,POINT (-74.01312 40.70244)
5,New York City Limo Service,40.745192,-73.983371,POINT (-73.98337 40.74519)
6,New Bell Car Service,40.693717,-73.960250,POINT (-73.96025 40.69372)
7,Times Square Limousine,40.752182,-74.003348,POINT (-74.00335 40.75218)
8,Charter Jet One,40.752684,-73.975902,POINT (-73.97590 40.75268)
9,Lower East Side Car Service,40.726151,-73.977334,POINT (-73.97733 40.72615)


In [23]:
#Map(Layer(gdf_aeropuertos,"color:blue",popup_hover = [popup_element("nombre", "Aeropuertos")]))

🔴  **¿Cual será el más cercano?**

In [24]:
#Transformamos la información en tipo "point"
gdf_aeropuertos['geometry']=gdf_aeropuertos['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_aeropuertos

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,nombre,latitud,longitud,geometry
0,PH Helipad,40.732918,-73.995468,"{'type': 'Point', 'coordinates': (-73.99546813..."
1,Corporate Express Travel,40.722429,-74.002404,"{'type': 'Point', 'coordinates': (-74.00240421..."
2,Priority One Jets,40.709288,-74.006822,"{'type': 'Point', 'coordinates': (-74.00682172..."
3,Atlantic Aviation,40.744048,-73.973925,"{'type': 'Point', 'coordinates': (-73.97392544..."
4,Us Helicopter,40.702437,-74.013122,"{'type': 'Point', 'coordinates': (-74.01312247..."
5,New York City Limo Service,40.745192,-73.983371,"{'type': 'Point', 'coordinates': (-73.98337125..."
6,New Bell Car Service,40.693717,-73.960250,"{'type': 'Point', 'coordinates': (-73.96025, 4..."
7,Times Square Limousine,40.752182,-74.003348,"{'type': 'Point', 'coordinates': (-74.00334835..."
8,Charter Jet One,40.752684,-73.975902,"{'type': 'Point', 'coordinates': (-73.9759022,..."
9,Lower East Side Car Service,40.726151,-73.977334,"{'type': 'Point', 'coordinates': (-73.977334, ..."


*Para realizar las oportunas geoqueries necesitamos crear una indexación geoespacial en MongoDB.*

In [25]:
client = MongoClient()
db = client["Ironhack-lab"]
collection1 = db.aeropuertos
collection1.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

Una vez creada la nueva colección introducimos los datos referentes a la lista de aeropuertos.

In [26]:
data1 = gdf_aeropuertos.to_dict(orient="records")
collection1.insert_many(data1)

Emplearemos la función la $near la cual nos indica el elemento más cercano en este caso a las coordenadas de la oficina.

*Como la lista no las da ordenada de más cercano a más lejano tomaremos solo el primer elemento*

In [27]:
cercano1 = db.aeropuertos.find({"geometry":{"$near":{"type":"Point", "coordinates":[40.726155, -73.995625]}}})
list(cercano1)[0]

{'_id': ObjectId('60d97f6fbaa8249db79ab578'),
 'nombre': 'New Bell Car Service',
 'latitud': 40.69371727,
 'longitud': -73.96025,
 'geometry': {'type': 'Point', 'coordinates': [-73.96025, 40.69371727]}}

Para calcular la distancia en km entre los dos puntos se empleará la fórmula de Harvesine implementada en una función a través de Python.

In [28]:
def distancia(lat1, lon1, lat2, lon2):
    rad=math.pi/180
    dlat=lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia

In [29]:
distancia(40.72628,-73.995625,40.69371727,-73.96025)

4.691816306762731

✅ Resumen: El más cercano es "New Bell Car Service" a 4,69 km.

## 4.2 Discotecas 💃

*Para realizar el estudio de esta variable y las siguientes se seguirá el mismo proceso que para obtener los datos relativos al aeropuerto.*

In [30]:
params2 = {"client_id" : client_id,
              "client_secret" : client_secret,
              "v": "20180323",
              "ll": coordenadas,
              "query": "nightclub",
              "limit": 10,
              "radius": 500
             }
response2 = requests.get(url,params=params2).json()

In [31]:
data2 = response2.get("response")
decode2 = data2.get("groups")[0] 
discoteca = decode2.get("items")

In [32]:
#Extración de los datos
discotecas = []
for dic in discoteca:
    lista = {}
    lista["nombre"] = getFromDict(dic, mapa_nombre)
    lista["latitud"] = getFromDict(dic, mapa_latitud)
    lista["longitud"] = getFromDict(dic, mapa_longitud)
    discotecas.append(lista)

In [33]:
#Creación del DataFrame
df_discotecas = pd.DataFrame(discotecas)
df_discotecas

,nombre,latitud,longitud
0,Public Arts,40.722948,-73.991726
1,ACME Basement,40.727362,-73.993846
2,Von,40.725300,-73.992508
3,8 Bond Street,40.726730,-73.994224
4,The Loft,40.726084,-73.996820
5,Le Poisson Rouge,40.728490,-73.999952
6,The Gallery at LPR,40.728409,-73.999977
7,Negril Village,40.729566,-73.998304
8,Carroll Place,40.728544,-73.999834
9,Terra Blues,40.728288,-73.999381


In [34]:
#Lo transformamos en GeoDataFrame
gdf_discotecas = gpd.GeoDataFrame(df_discotecas, geometry = gpd.points_from_xy(df_discotecas.longitud, df_discotecas.latitud))
gdf_discotecas

,nombre,latitud,longitud,geometry
0,Public Arts,40.722948,-73.991726,POINT (-73.99173 40.72295)
1,ACME Basement,40.727362,-73.993846,POINT (-73.99385 40.72736)
2,Von,40.725300,-73.992508,POINT (-73.99251 40.72530)
3,8 Bond Street,40.726730,-73.994224,POINT (-73.99422 40.72673)
4,The Loft,40.726084,-73.996820,POINT (-73.99682 40.72608)
5,Le Poisson Rouge,40.728490,-73.999952,POINT (-73.99995 40.72849)
6,The Gallery at LPR,40.728409,-73.999977,POINT (-73.99998 40.72841)
7,Negril Village,40.729566,-73.998304,POINT (-73.99830 40.72957)
8,Carroll Place,40.728544,-73.999834,POINT (-73.99983 40.72854)
9,Terra Blues,40.728288,-73.999381,POINT (-73.99938 40.72829)


In [35]:
#Map(Layer(gdf_discotecas,"color:pink",popup_hover = [popup_element("nombre", "Discotecas")]))

🔴 **¿Cual será la más cercano?** 

In [36]:
gdf_discotecas['geometry'] = gdf_discotecas['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_discotecas

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,nombre,latitud,longitud,geometry
0,Public Arts,40.722948,-73.991726,"{'type': 'Point', 'coordinates': (-73.99172649..."
1,ACME Basement,40.727362,-73.993846,"{'type': 'Point', 'coordinates': (-73.99384600..."
2,Von,40.725300,-73.992508,"{'type': 'Point', 'coordinates': (-73.99250765..."
3,8 Bond Street,40.726730,-73.994224,"{'type': 'Point', 'coordinates': (-73.994224, ..."
4,The Loft,40.726084,-73.996820,"{'type': 'Point', 'coordinates': (-73.99682, 4..."
5,Le Poisson Rouge,40.728490,-73.999952,"{'type': 'Point', 'coordinates': (-73.99995206..."
6,The Gallery at LPR,40.728409,-73.999977,"{'type': 'Point', 'coordinates': (-73.99997711..."
7,Negril Village,40.729566,-73.998304,"{'type': 'Point', 'coordinates': (-73.99830435..."
8,Carroll Place,40.728544,-73.999834,"{'type': 'Point', 'coordinates': (-73.99983388..."
9,Terra Blues,40.728288,-73.999381,"{'type': 'Point', 'coordinates': (-73.99938089..."


In [37]:
client = MongoClient()
db = client["Ironhack-lab"]
collection2 = db.discotecas
collection2.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [38]:
data2 = gdf_discotecas.to_dict(orient="records")
collection2.insert_many(data2)

In [39]:
cercano2 = db.discotecas.find({"geometry":{"$near":{"type":"Point", "coordinates":[40.726155, -73.995625],"$minDistance": 500, "$maxDistance":300}}})
list(cercano2)[0]

{'_id': ObjectId('60d97f98baa8249db79ab57d'),
 'nombre': 'Public Arts',
 'latitud': 40.72294781563586,
 'longitud': -73.99172649617472,
 'geometry': {'type': 'Point',
  'coordinates': [-73.99172649617472, 40.72294781563586]}}

In [40]:
distancia(40.72628,-73.995625,40.72294781563586,-73.99172649617472)

0.4953317318767005

✅ Resumen: La discoteca más cercana es "Public Arts" a unos 495 metros.

## 4.3 Starbucks ☕

In [41]:
params3 = {"client_id" : client_id,
              "client_secret" : client_secret,
              "v": "20180323",
              "ll": coordenadas,
              "query": "starbucks",
              "limit": 10,
              "radius": 500
             }
response3 = requests.get(url,params=params3).json()

In [42]:
data3 = response3.get("response")
decode3 = data3.get("groups")[0] 
starbuck = decode3.get("items")

In [43]:
#Extración de los datos
starbucks = []
for dic in starbuck:
    lista = {}
    lista["nombre"] = getFromDict(dic, mapa_nombre)
    lista["latitud"] = getFromDict(dic, mapa_latitud)
    lista["longitud"] = getFromDict(dic, mapa_longitud)
    starbucks.append(lista)

In [44]:
df_starbucks = pd.DataFrame(starbucks)
df_starbucks

,nombre,latitud,longitud
0,Starbucks Reserve,40.729943,-73.993913
1,Starbucks,40.727192,-73.995460
2,Starbucks,40.726890,-74.000260
3,Starbucks,40.722585,-73.998023
4,Starbucks,40.729655,-73.996179


In [45]:
#Lo transformamos en GeoDataFrame
gdf_starbucks = gpd.GeoDataFrame(df_starbucks, geometry = gpd.points_from_xy(df_starbucks.longitud, df_starbucks.latitud))
gdf_starbucks

,nombre,latitud,longitud,geometry
0,Starbucks Reserve,40.729943,-73.993913,POINT (-73.99391 40.72994)
1,Starbucks,40.727192,-73.995460,POINT (-73.99546 40.72719)
2,Starbucks,40.726890,-74.000260,POINT (-74.00026 40.72689)
3,Starbucks,40.722585,-73.998023,POINT (-73.99802 40.72258)
4,Starbucks,40.729655,-73.996179,POINT (-73.99618 40.72966)


In [46]:
#Map(Layer(gdf_starbucks,"color:darkgreen",popup_hover = [popup_element("nombre", "Starbucks")]))

🔴 La empresa dispone de 5 Starbucks cercanos en un radio de 500 metros. Sin embargo de todos ellos:  
**¿Cual será el más cercano?** 

In [47]:
gdf_starbucks['geometry'] = gdf_starbucks['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_starbucks

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,nombre,latitud,longitud,geometry
0,Starbucks Reserve,40.729943,-73.993913,"{'type': 'Point', 'coordinates': (-73.99391319..."
1,Starbucks,40.727192,-73.995460,"{'type': 'Point', 'coordinates': (-73.99546, 4..."
2,Starbucks,40.726890,-74.000260,"{'type': 'Point', 'coordinates': (-74.00026, 4..."
3,Starbucks,40.722585,-73.998023,"{'type': 'Point', 'coordinates': (-73.99802280..."
4,Starbucks,40.729655,-73.996179,"{'type': 'Point', 'coordinates': (-73.996179, ..."


In [48]:
client = MongoClient()
db = client["Ironhack-lab"]
collection3 = db.starbucks
collection3.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [49]:
data3 = gdf_starbucks.to_dict(orient="records")
collection3.insert_many(data3)

In [50]:
cercano3 = db.starbucks.find({"geometry":{"$near":{"type":"Point", "coordinates":[40.726155, -73.995625],"$minDistance": 500, "$maxDistance":300}}})
list(cercano3)[0]

{'_id': ObjectId('60d97fb8baa8249db79ab58b'),
 'nombre': 'Starbucks',
 'latitud': 40.722584953691005,
 'longitud': -73.9980228047693,
 'geometry': {'type': 'Point',
  'coordinates': [-73.9980228047693, 40.722584953691005]}}

In [51]:
distancia(40.72628,-73.995625,40.722584953691005,-73.9980228047693)

0.4579977064355368

✅ Resumen: El Starbucks más cercano se encuentra a 458 metros

## 4.4 Restaurante Vegano 🥕

In [52]:
params4 = {"client_id" : client_id,
              "client_secret" : client_secret,
              "v": "20180323",
              "ll": coordenadas,
              "query": "vegan restaurants",
              "limit": 10,
              "radius": 500
             }
response4 = requests.get(url,params=params4).json()

In [53]:
data4 = response4.get("response")
decode4 = data4.get("groups")[0] 
vegan = decode4.get("items")

In [54]:
#Extración de los datos
resveganos = []
for dic in vegan:
    lista = {}
    lista["nombre"] = getFromDict(dic, mapa_nombre)
    lista["latitud"] = getFromDict(dic, mapa_latitud)
    lista["longitud"] = getFromDict(dic, mapa_longitud)
    resveganos.append(lista)

In [55]:
#Creación del DataFrame
df_resveganos = pd.DataFrame(resveganos)
df_resveganos

,nombre,latitud,longitud
0,Little Atlas,40.728646,-73.994723
1,by CHLOE.,40.722770,-73.997200
2,Juice Press,40.723439,-73.994877
3,Farmacy At Chef’s Club Counter,40.722263,-73.997388
4,Quantum Leap,40.729209,-73.998867
5,Double Zero,40.726108,-73.989806
6,Beyond Sushi,40.722080,-73.996250
7,Bar Verde,40.726064,-73.989779
8,Juice Press,40.725941,-74.001189
9,&pizza,40.729770,-73.992736


In [56]:
#Lo transformamos en GeoDataFrame
gdf_resveganos = gpd.GeoDataFrame(df_resveganos, geometry = gpd.points_from_xy(df_resveganos.longitud, df_resveganos.latitud))
gdf_resveganos

,nombre,latitud,longitud,geometry
0,Little Atlas,40.728646,-73.994723,POINT (-73.99472 40.72865)
1,by CHLOE.,40.722770,-73.997200,POINT (-73.99720 40.72277)
2,Juice Press,40.723439,-73.994877,POINT (-73.99488 40.72344)
3,Farmacy At Chef’s Club Counter,40.722263,-73.997388,POINT (-73.99739 40.72226)
4,Quantum Leap,40.729209,-73.998867,POINT (-73.99887 40.72921)
5,Double Zero,40.726108,-73.989806,POINT (-73.98981 40.72611)
6,Beyond Sushi,40.722080,-73.996250,POINT (-73.99625 40.72208)
7,Bar Verde,40.726064,-73.989779,POINT (-73.98978 40.72606)
8,Juice Press,40.725941,-74.001189,POINT (-74.00119 40.72594)
9,&pizza,40.729770,-73.992736,POINT (-73.99274 40.72977)


In [57]:
#Map(Layer(gdf_resveganos,"color:green",popup_hover = [popup_element("nombre", "Restaurantes Veganos")]))

🔴 **¿Cual será el más cercano?** 

In [58]:
gdf_resveganos['geometry'] = gdf_resveganos['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_resveganos

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,nombre,latitud,longitud,geometry
0,Little Atlas,40.728646,-73.994723,"{'type': 'Point', 'coordinates': (-73.99472251..."
1,by CHLOE.,40.722770,-73.997200,"{'type': 'Point', 'coordinates': (-73.9972, 40..."
2,Juice Press,40.723439,-73.994877,"{'type': 'Point', 'coordinates': (-73.99487685..."
3,Farmacy At Chef’s Club Counter,40.722263,-73.997388,"{'type': 'Point', 'coordinates': (-73.997388, ..."
4,Quantum Leap,40.729209,-73.998867,"{'type': 'Point', 'coordinates': (-73.99886686..."
5,Double Zero,40.726108,-73.989806,"{'type': 'Point', 'coordinates': (-73.98980624..."
6,Beyond Sushi,40.722080,-73.996250,"{'type': 'Point', 'coordinates': (-73.99625027..."
7,Bar Verde,40.726064,-73.989779,"{'type': 'Point', 'coordinates': (-73.9897793,..."
8,Juice Press,40.725941,-74.001189,"{'type': 'Point', 'coordinates': (-74.00118930..."
9,&pizza,40.729770,-73.992736,"{'type': 'Point', 'coordinates': (-73.99273619..."


In [59]:
client = MongoClient()
db = client["Ironhack-lab"]
collection4 = db.resveganos
collection4.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [60]:
data4 = gdf_resveganos.to_dict(orient="records")
collection4.insert_many(data4)

In [61]:
cercano4 = db.resveganos.find({"geometry":{"$near":{"type":"Point", "coordinates":[40.726155, -73.995625],"$minDistance": 500, "$maxDistance":300}}})
list(cercano4)[0]

{'_id': ObjectId('60d97fdbbaa8249db79ab594'),
 'nombre': 'Beyond Sushi',
 'latitud': 40.72207953581667,
 'longitud': -73.99625027875686,
 'geometry': {'type': 'Point',
  'coordinates': [-73.99625027875686, 40.72207953581667]}}

In [62]:
distancia(40.72628,-73.995625,40.72207953581667,-73.99625027875686)

0.4701656012037206

✅ Resumen: El restaurante vegano más cercano se encuentra a 470 metros

## 4.5 Estación de autobuses 🚌

In [63]:
params5 = {"client_id" : client_id,
              "client_secret" : client_secret,
              "v": "20180323",
              "ll": coordenadas,
              "query": "bus station",
              "limit": 10,
              "radius": 500
             }
response5 = requests.get(url,params=params5).json()

In [64]:
data5 = response5.get("response")
decode5 = data5.get("groups")[0] 
bus = decode5.get("items")

In [65]:
#Extración de los datos
autobuses = []
for dic in bus:
    lista = {}
    lista["nombre"] = getFromDict(dic, mapa_nombre)
    lista["latitud"] = getFromDict(dic, mapa_latitud)
    lista["longitud"] = getFromDict(dic, mapa_longitud)
    autobuses.append(lista)

In [66]:
#Creación del DataFrame
df_autobuses = pd.DataFrame(autobuses)
df_autobuses

,nombre,latitud,longitud
0,"MTA X7, X9, X10, X27B, X28B (Houston St/Mercer...",40.725877,-73.997340
1,M21 bus stop,40.726785,-73.998616
2,MTA Bus 103,40.725060,-73.992280
3,M21 Bus BW & HS,40.725313,-73.996228
4,OvRride Bus to Windham,40.727263,-73.995824
5,Bleecker St - Skyline Bus Tour,40.726576,-73.995903
6,MTA Bus Stop M5 @ Broadway And Houston,40.725447,-73.997169
7,MTA Bus - E Houston St & Crosby St (M21),40.725409,-73.996106
8,MTA Bus - E Houston St & Broadway (M21),40.725141,-73.996310
9,NYU Shuttle - Route A,40.728766,-73.993848


In [67]:
#Lo transformamos en GeoDataFrame
gdf_autobuses = gpd.GeoDataFrame(df_autobuses, geometry = gpd.points_from_xy(df_autobuses.longitud, df_autobuses.latitud))
gdf_autobuses

,nombre,latitud,longitud,geometry
0,"MTA X7, X9, X10, X27B, X28B (Houston St/Mercer...",40.725877,-73.997340,POINT (-73.99734 40.72588)
1,M21 bus stop,40.726785,-73.998616,POINT (-73.99862 40.72679)
2,MTA Bus 103,40.725060,-73.992280,POINT (-73.99228 40.72506)
3,M21 Bus BW & HS,40.725313,-73.996228,POINT (-73.99623 40.72531)
4,OvRride Bus to Windham,40.727263,-73.995824,POINT (-73.99582 40.72726)
5,Bleecker St - Skyline Bus Tour,40.726576,-73.995903,POINT (-73.99590 40.72658)
6,MTA Bus Stop M5 @ Broadway And Houston,40.725447,-73.997169,POINT (-73.99717 40.72545)
7,MTA Bus - E Houston St & Crosby St (M21),40.725409,-73.996106,POINT (-73.99611 40.72541)
8,MTA Bus - E Houston St & Broadway (M21),40.725141,-73.996310,POINT (-73.99631 40.72514)
9,NYU Shuttle - Route A,40.728766,-73.993848,POINT (-73.99385 40.72877)


In [68]:
#Map(Layer(gdf_autobuses,"color:purple",popup_hover = [popup_element("nombre", "Estación de autobuses")]))

🔴 **¿Cual será el más cercano?** 

In [69]:
gdf_autobuses['geometry'] = gdf_autobuses['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_autobuses

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,nombre,latitud,longitud,geometry
0,"MTA X7, X9, X10, X27B, X28B (Houston St/Mercer...",40.725877,-73.997340,"{'type': 'Point', 'coordinates': (-73.99734020..."
1,M21 bus stop,40.726785,-73.998616,"{'type': 'Point', 'coordinates': (-73.99861564..."
2,MTA Bus 103,40.725060,-73.992280,"{'type': 'Point', 'coordinates': (-73.99228003..."
3,M21 Bus BW & HS,40.725313,-73.996228,"{'type': 'Point', 'coordinates': (-73.99622821..."
4,OvRride Bus to Windham,40.727263,-73.995824,"{'type': 'Point', 'coordinates': (-73.995824, ..."
5,Bleecker St - Skyline Bus Tour,40.726576,-73.995903,"{'type': 'Point', 'coordinates': (-73.99590254..."
6,MTA Bus Stop M5 @ Broadway And Houston,40.725447,-73.997169,"{'type': 'Point', 'coordinates': (-73.99716949..."
7,MTA Bus - E Houston St & Crosby St (M21),40.725409,-73.996106,"{'type': 'Point', 'coordinates': (-73.99610638..."
8,MTA Bus - E Houston St & Broadway (M21),40.725141,-73.996310,"{'type': 'Point', 'coordinates': (-73.99631023..."
9,NYU Shuttle - Route A,40.728766,-73.993848,"{'type': 'Point', 'coordinates': (-73.993848, ..."


In [70]:
client = MongoClient()
db = client["Ironhack-lab"]
collection5 = db.autobuses
collection5.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [71]:
data5 = gdf_autobuses.to_dict(orient="records")
collection5.insert_many(data5)

In [72]:
cercano5 = db.autobuses.find({"geometry":{"$near":{"type":"Point", "coordinates":[40.726155, -73.995625],"$minDistance": 500, "$maxDistance":300}}})
list(cercano5)[0]

{'_id': ObjectId('60d97fffbaa8249db79ab59b'),
 'nombre': 'MTA Bus 103',
 'latitud': 40.72506034424643,
 'longitud': -73.99228003980646,
 'geometry': {'type': 'Point',
  'coordinates': [-73.99228003980646, 40.72506034424643]}}

In [73]:
distancia(40.72628,-73.995625,40.72506034424643,-73.99228003980646)

0.31289076969772284

✅ Resumen: La parada de autobuses más cercana se encuentra a 312 metros

## 4.6 Colegios 👶🏻

In [74]:
params6 = {"client_id" : client_id,
              "client_secret" : client_secret,
              "v": "20180323",
              "ll": coordenadas,
              "query": "school",
              "limit": 10,
              "radius": 500
             }
response6 = requests.get(url,params=params6).json() 

In [75]:
data6 = response6.get("response")
decode6 = data6.get("groups")[0] 
escuela = decode6.get("items")

In [76]:
#Extración de los datos
escuelas = []
for dic in escuela:
    lista = {}
    lista["nombre"] = getFromDict(dic, mapa_nombre)
    lista["latitud"] = getFromDict(dic, mapa_latitud)
    lista["longitud"] = getFromDict(dic, mapa_longitud)
    escuelas.append(lista)

In [77]:
#Creación del DataFrame
df_escuelas = pd.DataFrame(escuelas)
df_escuelas

,nombre,latitud,longitud
0,AppAcademy,40.725326,-73.996808
1,MUJI,40.728645,-73.991321
2,Kaleidoscope Kids,40.728154,-73.997730
3,Make Up For Ever Academy,40.723749,-73.998006
4,Skillshare School,40.722672,-73.995155
5,The Hemispheric Institute of Performance and P...,40.727893,-73.991590
6,Kimmel 604 - N'Harmz!,40.730061,-73.997726
7,La Salle Academy,40.724944,-73.990278
8,Kaplan International English,40.725790,-73.996446
9,NYU Wagner - Robert F. Wagner Graduate School ...,40.724804,-73.995343


In [78]:
#Lo transformamos en GeoDataFrame
gdf_escuelas = gpd.GeoDataFrame(df_escuelas, geometry = gpd.points_from_xy(df_escuelas.longitud, df_escuelas.latitud))
gdf_escuelas

,nombre,latitud,longitud,geometry
0,AppAcademy,40.725326,-73.996808,POINT (-73.99681 40.72533)
1,MUJI,40.728645,-73.991321,POINT (-73.99132 40.72865)
2,Kaleidoscope Kids,40.728154,-73.997730,POINT (-73.99773 40.72815)
3,Make Up For Ever Academy,40.723749,-73.998006,POINT (-73.99801 40.72375)
4,Skillshare School,40.722672,-73.995155,POINT (-73.99516 40.72267)
5,The Hemispheric Institute of Performance and P...,40.727893,-73.991590,POINT (-73.99159 40.72789)
6,Kimmel 604 - N'Harmz!,40.730061,-73.997726,POINT (-73.99773 40.73006)
7,La Salle Academy,40.724944,-73.990278,POINT (-73.99028 40.72494)
8,Kaplan International English,40.725790,-73.996446,POINT (-73.99645 40.72579)
9,NYU Wagner - Robert F. Wagner Graduate School ...,40.724804,-73.995343,POINT (-73.99534 40.72480)


In [79]:
#Map(Layer(gdf_escuelas,"color:orange",popup_hover = [popup_element("nombre", "Escuelas")]))

🔴  **¿Cual será el más cercano?** 

In [80]:
gdf_escuelas['geometry'] = gdf_escuelas['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_escuelas

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,nombre,latitud,longitud,geometry
0,AppAcademy,40.725326,-73.996808,"{'type': 'Point', 'coordinates': (-73.99680795..."
1,MUJI,40.728645,-73.991321,"{'type': 'Point', 'coordinates': (-73.99132132..."
2,Kaleidoscope Kids,40.728154,-73.997730,"{'type': 'Point', 'coordinates': (-73.99773039..."
3,Make Up For Ever Academy,40.723749,-73.998006,"{'type': 'Point', 'coordinates': (-73.9980063,..."
4,Skillshare School,40.722672,-73.995155,"{'type': 'Point', 'coordinates': (-73.99515533..."
5,The Hemispheric Institute of Performance and P...,40.727893,-73.991590,"{'type': 'Point', 'coordinates': (-73.99158954..."
6,Kimmel 604 - N'Harmz!,40.730061,-73.997726,"{'type': 'Point', 'coordinates': (-73.99772614..."
7,La Salle Academy,40.724944,-73.990278,"{'type': 'Point', 'coordinates': (-73.99027751..."
8,Kaplan International English,40.725790,-73.996446,"{'type': 'Point', 'coordinates': (-73.99644625..."
9,NYU Wagner - Robert F. Wagner Graduate School ...,40.724804,-73.995343,"{'type': 'Point', 'coordinates': (-73.99534254..."


In [81]:
client = MongoClient()
db = client["Ironhack-lab"]
collection6 = db.escuelas
collection6.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [82]:
data6 = gdf_escuelas.to_dict(orient="records")
collection6.insert_many(data6)

In [83]:
cercano6 = db.escuelas.find({"geometry":{"$near":{"type":"Point", "coordinates":[40.726155, -73.995625],"$minDistance": 500, "$maxDistance":300}}})
list(cercano6)[0]

{'_id': ObjectId('60d9801ebaa8249db79ab5a8'),
 'nombre': 'Skillshare School',
 'latitud': 40.72267150878906,
 'longitud': -73.99515533447266,
 'geometry': {'type': 'Point',
  'coordinates': [-73.99515533447266, 40.72267150878906]}}

In [84]:
distancia(40.72628,-73.995625,40.72267150878906,-73.99515533447266)

0.4033068180917176

✅ Resumen: El colegio más cercano se encuentra a 403 metros.

## 4.7 Veterinario 🐾

In [85]:
params7 = {"client_id" : client_id,
              "client_secret" : client_secret,
              "v": "20180323",
              "ll": coordenadas,
              "query": "veterinary",
              "limit": 10,
              "radius": 500
             }
response7 = requests.get(url,params=params7).json() 

In [86]:
data7 = response7.get("response")
decode7 = data7.get("groups")[0] 
vet = decode7.get("items")

In [87]:
#Extración de los datos
veterinario = []
for dic in vet:
    lista = {}
    lista["nombre"] = getFromDict(dic, mapa_nombre)
    lista["latitud"] = getFromDict(dic, mapa_latitud)
    lista["longitud"] = getFromDict(dic, mapa_longitud)
    veterinario.append(lista)

In [88]:
#Creación del DataFrame
df_veterinario = pd.DataFrame(veterinario)
df_veterinario

,nombre,latitud,longitud
0,Cooper Square Veterinary Hospital,40.727466,-73.990615


In [89]:
#Lo transformamos en GeoDataFrame
gdf_veterinario = gpd.GeoDataFrame(df_veterinario, geometry = gpd.points_from_xy(df_veterinario.longitud, df_veterinario.latitud))
gdf_veterinario

,nombre,latitud,longitud,geometry
0,Cooper Square Veterinary Hospital,40.727466,-73.990615,POINT (-73.99061 40.72747)


In [90]:
#Map(Layer(gdf_veterinario,"color:black",popup_hover = [popup_element("nombre", "Veterinarios")]))

🔴  **Distancia en metros** 

In [91]:
gdf_veterinario['geometry'] = gdf_veterinario['geometry'].apply(lambda x:shapely.geometry.mapping(x))
gdf_veterinario

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,nombre,latitud,longitud,geometry
0,Cooper Square Veterinary Hospital,40.727466,-73.990615,"{'type': 'Point', 'coordinates': (-73.99061496..."


In [92]:
client = MongoClient()
db = client["Ironhack-lab"]
collection7 = db.veterinario
collection7.create_index ([("geometry", GEOSPHERE)])

'geometry_2dsphere'

In [93]:
data7 = gdf_veterinario.to_dict(orient="records")
collection7.insert_many(data7)

In [94]:
#cercano7 = db.veterinario.find({"geometry":{"$near":{"type":"Point", "coordinates":[40.726155, -73.995625],"$minDistance": 500, "$maxDistance":300}}})
#list(cercano7)[0]

In [95]:
distancia(40.72628,-73.995625,40.72746618943346,-73.99061496749717)

0.44242778677806777

✅ Resumen: El veterinario más cercano se encuentra a 442 metros.